<a href="https://colab.research.google.com/github/strawndri/corretor_ortografico-nlp/blob/main/Colaboratory/Corretor_ortografico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP - Processamento de Linguagem Natural

A **Processamento de Linguagem Natural (NLP, em inglês)** está relacionada à Inteligência Antificial. Este mecanismo, portanto, atua na conexão entre o ser humano e a máquina.

A interpretação de textos realizada por pessoas, é de certa forma, simples. No entanto, quando este papel cabe à máquina, a situação torna-se mais complicada, uma vez que ela precisa "compreender" inúmeros peculiaridades linguísticas, como coensão, coêrencia, morfologia, semântica, sintaxe, etc.

Alguns exemplos de atividades comuns que utlizam o NLP são: tradução automática, corretor ortográfico, autocomplete (realizado pelo Google, por exemplo), chatbots...

# Importações

In [32]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Importando um Corpus Textual
Obs: Corpus significa, em NLP, uma coleção de documentos

In [33]:
with open('artigos.txt', 'r') as f:
  artigos = f.read()

print(artigos[:500])




imagem 

Temos a seguinte classe que representa um usuário no nosso sistema:

java

Para salvar um novo usuário, várias validações são feitas, como por exemplo: Ver se o nome só contém letras, [**o CPF só números**] e ver se o usuário possui no mínimo 18 anos. Veja o método que faz essa validação:

java 

Suponha agora que eu tenha outra classe, a classe `Produto`, que contém um atributo nome e eu quero fazer a mesma validação que fiz para o nome do usuário: Ver se só contém letras. E aí? Vou


# Tokens

Os **tokens** são uma sequência de caracteres (podem conter letras, números, pontuações, etc) separados por um limitador, que pode ser tanto um espaço em branco, como uma quebra de linha ou determinada pontuação).

**Tozenização** é um processo que divide strings em partes menores.

In [34]:
tokens = artigos.split()
print(f'Quantidade de tokens: {len(tokens)}')

Quantidade de tokens: 416903


In [35]:
tokens_2 = nltk.tokenize.word_tokenize(artigos)

In [36]:
print(tokens_2[:50])

['imagem', 'Temos', 'a', 'seguinte', 'classe', 'que', 'representa', 'um', 'usuário', 'no', 'nosso', 'sistema', ':', 'java', 'Para', 'salvar', 'um', 'novo', 'usuário', ',', 'várias', 'validações', 'são', 'feitas', ',', 'como', 'por', 'exemplo', ':', 'Ver', 'se', 'o', 'nome', 'só', 'contém', 'letras', ',', '[', '**o', 'CPF', 'só', 'números**', ']', 'e', 'ver', 'se', 'o', 'usuário', 'possui', 'no']


In [37]:
def separa_palavras(lista_tokens):
  lista_palavras = []

  for item in lista_tokens:
    if item.isalpha():
      lista_palavras.append(item)

  return lista_palavras    

palavras = separa_palavras(tokens_2)
print(f'Total de palavras: {len(palavras)}')

Total de palavras: 393914


# Normalização e Tipos de Palavras

In [38]:
def normaliza_palavras(lista):
  lista_normalizada = []

  for item in lista:
    lista_normalizada.append(item.lower())

  return lista_normalizada

lista_normalizada = normaliza_palavras(palavras)
print(lista_normalizada[:10])

['imagem', 'temos', 'a', 'seguinte', 'classe', 'que', 'representa', 'um', 'usuário', 'no']


In [39]:
total_palavras = set(lista_normalizada)
print(f'Total de palavras diferentes: {len(total_palavras)}.')

Total de palavras diferentes: 17652.


# Desenvolvimento do Corretor

## Ações 

In [40]:
# função que insere as possíveis palavras
def insere_palavras(slices):

  novas_palavras = []
  conjunto_letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'

  for left, right in slices:
    for l in conjunto_letras:
      novas_palavras.append(left + l + right)
  
  return novas_palavras

In [41]:
#função que configura a palavra incorreta para correta deletando caracteres
def deleta_caracteres(slices):
  novas_palavras = []

  for left, right in slices:
    novas_palavras.append(left + right[1:])
  
  return novas_palavras

In [42]:
# função que aletar caracteres 
def altera_caracteres(slices):
  novas_palavras = []
  conjunto_letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'

  for left, right in slices:
    for l in conjunto_letras:
      novas_palavras.append(left + l + right[1:])
  
  return novas_palavras

In [43]:
# função que inverte a ordem de caracteres
def inverte_caracteres(slices):
  novas_palavras = []

  for left, right in slices:
    if (len(right) > 1):
      novas_palavras.append(left + right[1] + right[0] + right[2])
  
  return novas_palavras

In [44]:
# função que constrói os trechos de uma palavra
def gerador_palavras(palavra):

  slices = []

  for i in range(len(palavra) + 1):
    slices.append(((palavra[:i]), (palavra[i:])))

  palavras_geradas = insere_palavras(slices)
  palavras_geradas += deleta_caracteres(slices)
  palavras_geradas += altera_caracteres(slices)
  return palavras_geradas

## Corretor

In [45]:
# função que atua como corretor ortográfico
def corretor(palavra):
  palavras_geradas = gerador_palavras(palavra)
  palavra_correta = max(palavras_geradas, key=propabilidade)
  return palavra_correta


freq = nltk.FreqDist(lista_normalizada)
total_palavras = len(palavras)

def propabilidade(palavra_gerada):
  probabilidade = freq[palavra_gerada]/total_palavras
  return probabilidade

## Avaliação

In [46]:
# Função que cria uma lista com palavras-exemplo
def cria_testes(arquivo):
  lista_teste = []
  f = open(arquivo, "r")

  for linha in f:
    correta, incorreta = linha.split()
    lista_teste.append((correta, incorreta))
  f.close()

  return lista_teste

lista_teste = cria_testes("palavras.txt")

In [47]:
# função que retorna a taxa de acertos do corretor
def avalia_corretor(testes):
  total_palavras = len(testes)
  acertou = 0

  # corrige a palavra incorreta
  for correta, incorreta in testes:
    palavra_corrigida = corretor(incorreta)
    if (palavra_corrigida == correta):
      acertou += 1

  taxa_de_acerto = round((acertou * 100) / total_palavras, 2)
  
  print(f'Taxa de acerto de {taxa_de_acerto}% em {total_palavras} palavras-teste.')
  
avalia_corretor(lista_teste)

Taxa de acerto de 76.34% em 186 palavras-teste.
